In [ ]:
# default_exp data

# data
> Provides dataset for icevision records and classmaps

In [ ]:
#export
import datetime
from typing import Union, Optional, List
import os
import shutil
import json

import pytest

import numpy as np
import pandas as pd

from icevision.core.record import BaseRecord
import icevision.parsers as parsers
from icevision.data.data_splitter import RandomSplitter
from icevision.core.bbox import BBox
from icevision.core.class_map import ClassMap

from icevision_dashboards.core.plotting.utils import draw_record_with_bokeh
from icevision_dashboards.core.data import *

## Test data setup

In [ ]:
import icedata

In [ ]:
test_data_dir = icedata.fridge.load_data()
test_class_map = icedata.fridge.class_map()
test_parser = icedata.fridge.parser(test_data_dir, test_class_map)
test_train_records, test_valid_records = test_parser.parse()

In [ ]:
#export
class RecordDataframeParser(parsers.Parser, parsers.FilepathMixin, parsers.SizeMixin, parsers.LabelsMixin):
    def __init__(self, record_dataframe):
        self.record_dataframe = record_dataframe
        
    def __iter__(self):
        for group in self.record_dataframe.groupby("filepath"):
            yield group[1]
    
    def imageid(self, o):
        return o.iloc[0]["id"]
    
    def filepath(self, o):
        return o.iloc[0]["filepath"]
    
    def image_width_height(self, o):
        width, height = o.iloc[0]["width"], o.iloc[0]["height"]
        return (width, height)
    
    def labels(self, o):
        return [annot[1]["label_num"] for annot in o.iterrows()]

In [ ]:
#export
class BboxRecordDataframeParser(RecordDataframeParser, parsers.BBoxesMixin):
    def bboxes(self, o):
        return [BBox(annot[1]["bbox_xmin"], annot[1]["bbox_ymin"], annot[1]["bbox_xmax"], annot[1]["bbox_ymax"]) for annot in o.iterrows()]

In [ ]:
#export
class MaskRecordDataframeParser(BboxRecordDataframeParser, parsers.MaskRCNN):
    def iscrowds(self, o):
        raise NotImplementedError()
    def masks(self, o):
        raise NotImplementedError()

In [ ]:
#export
class RecordDataset(GenericDataset):
    def __init__(self, records: Union[List[BaseRecord], ObservableList, str], class_map=None, name=None, description=None):
        if isinstance(records, str):
            self.load_from_file(records)
        else:
            self.records = records if isinstance(records, ObservableList) else ObservableList(records)
            self.class_map = class_map
        super().__init__(self.records, name=name, description=description)
        self.records.register_callback(self.reset_infered_data)
    
    def __repr__(self):
        base_string = ""
        for col in self.stats_dataset.columns:
            base_string += str(col) + ": " + str(self.stats_dataset[col][0]) + " | "
        base_string = base_string[:-2]
        return base_string
        
    def __getitem__(self, index):
        return self.records[index]
        
    def __len__(self):
        return len(self.records)
        
    @classmethod
    def load_from_record_dataframe(cls, record_data_df: pd.DataFrame, class_map=None, name=None, description=None):
        records = cls.parse_df_to_records(record_data_df)
        if class_map is None:
            class_map = cls.create_class_map_from_record_df(record_data_df)
        return cls(records, class_map=class_map, name=name, description=description)

    @staticmethod
    def create_class_map_from_record_df(record_df):
        sorted_labels = record_df["label"].unique()[np.argsort(record_df["label_num"].unique())].tolist()
        sorted_label_nums = sorted(record_df["label_num"].unique())
        label_map = {key: value for key, value in zip(sorted_label_nums, sorted_labels)}
        return ClassMap([label_map[i] if i in label_map.keys() else "unknown_"+str(i) for i in range(max(sorted_label_nums))])
    
    @staticmethod
    def parse_df_to_records(record_data_df):
        raise NotImplementedError()
    
    def load_from_file(self, path):
        data = json.load(open(path))
        df = pd.DataFrame(data["data"])
        records = self.parse_df_to_records(df)
        
        self.records = ObservableList(records)
        self.class_map = ClassMap(data["class_map"])
        self._name = data["name"]
        self._description = data["description"]
    
    def save(self, save_path):
        if not os.path.isdir(save_path):
            os.makedirs(save_path, exist_ok=True)
        save_name = "dataset" if self.name == "" else self.name
        if not os.path.isfile(os.path.join(save_path, save_name+".json")):
            save_name = save_name+".json"
        else:
            counter = 1
            while True:
                save_name = save_name+"("+str(counter)+").json"
                if os.path.isfile(os.path.join(save_path, save_name)):
                    counter += 1
                else:
                    break
        
        class_map = self.class_map if self.class_map is not None else self.create_class_map_from_record_df(df)
        save_data = {"name": self.name, "description": self.description, "data": self.data.to_dict(), "class_map": class_map.id2class}
        
        json.dump(save_data, open(os.path.join(save_path, save_name), "w"), default=str)
        
    def get_image_plot_by_id(self, index, width, height):
        return draw_record_with_bokeh(self[index], width=width, height=height, return_figure=True)
    
    @classmethod
    def create_new_from_mask(cls, cls_instance, mask):
        selection = cls_instance.data[mask]
        filepaths = np.unique(selection["filepath"]).tolist()
        new_records = [record for record in cls_instance.records if str(record["filepath"]) in filepaths]
        return cls(new_records)

In [ ]:
#export
class DataDescriptorBbox(DatasetDescriptor):
    def calculate_description(self, obj):
        """Aggregates stats from a list of records and returns a pandas dataframe with the aggregated stats. The creation time is not necessarily the real creation time. 
        This depends on the OS, for more information see: https://docs.python.org/3/library/os.html#os.stat_result."""
        data = []
        for index,record in enumerate(obj.records):
            for label, bbox in zip(record["labels"], record["bboxes"]):
                file_stats = record["filepath"].stat()
                bbox_widht = bbox.xmax - bbox.xmin
                bbox_height = bbox.ymax - bbox.ymin
                area = bbox_widht * bbox_height
                area_normalized = area / (record["width"] * record["height"])
                bbox_ratio = bbox_widht / bbox_height
                data.append(
                    {
                        "id": record["imageid"], "width": record["width"], "height": record["height"], "label": label, 
                        "bbox_xmin": bbox.xmin, "bbox_xmax": bbox.xmax, "bbox_ymin": bbox.ymin, "bbox_ymax": bbox.ymax, "area": area, 
                        "area_normalized": area_normalized, "bbox_ratio": bbox_ratio, "record_index": index, "bbox_width": bbox_widht, 
                        "bbox_height": bbox_height, "filepath": str(record["filepath"]), "creation_date": datetime.datetime.fromtimestamp(file_stats.st_ctime), 
                        "modification_date": datetime.datetime.fromtimestamp(file_stats.st_mtime), "num_annotations": len(record["bboxes"])
                    }
                )
        data = pd.DataFrame(data)
        data["label_num"] = data["label"]
        if obj.class_map is not None:
            data["label"] = data["label"].apply(obj.class_map.get_id)
        return data

In [ ]:
#export
class StatsDesscriptorBbox(DatasetDescriptor):
    def calculate_description(self, obj):
        stats_dict = {}
        stats_dict["no_imgs"] = [obj.data["filepath"].nunique()]
        stats_dict["no_classes"] = [obj.data["label"].nunique()]
        stats_dict["classes"] = [list(obj.data["label"].unique())]
        stats_dict["area_min"] = [obj.data["area"].min()]
        stats_dict["area_max"] = [obj.data["area"].max()]
        stats_dict["num_annotations_min"] = [obj.data["num_annotations"].min()]
        stats_dict["num_annotations_max"] = [obj.data["num_annotations"].max()]
        stats_dict["name"] = [obj._name]
        stats_dict["description"] = [obj._description]
        return pd.DataFrame(stats_dict)

In [ ]:
#export
class ImageStatsDesscriptorBbox(DatasetDescriptor):
    def calculate_description(self, obj):
        """Creates a dataframe containing stats about the images."""
        stats_dict = {}
        stats_dict["Num. imgs."] = obj.data["filepath"].nunique()
        stats_dict["Min Num. Objects"] = obj.data["num_annotations"].min()
        stats_dict["Max Num. Objects"] = obj.data["num_annotations"].max()
        stats_dict["Avg. Objects/Img"] = round(obj.data["num_annotations"].mean(),2)
        df = pd.DataFrame.from_dict(stats_dict, orient="index").T
        return df

In [ ]:
#export
class ClassStatsDesscriptorBbox(DatasetDescriptor):
    def calculate_description(self, obj):
        """Creates a dataframe containing stats about the object classes."""
        stats_dict = {}
        label_group = obj.data.groupby("label")
        for label, group in label_group:
            label_stats = {}
            label_stats["imgs"] = group["filepath"].nunique()
            label_stats["objects"] = group.shape[0]
            label_stats["avg_objects_per_img"] = label_stats["objects"]/label_stats["imgs"]
            label_stats["frac_of_labels"] = round(label_stats["objects"]/obj.data.shape[0], 2)
            stats_dict[label] = label_stats
        df = pd.DataFrame(stats_dict).T
        df = df.rename_axis('Class').reset_index()
        return df

In [ ]:
#export
class BboxRecordDataset(RecordDataset):
    data = DataDescriptorBbox()
    stats_dataset = StatsDesscriptorBbox()
    stats_image = ImageStatsDesscriptorBbox()
    stats_class = ClassStatsDesscriptorBbox()
    stats = StatsDesscriptorBbox()
    
    def __init__(self, records: Union[List[BaseRecord], ObservableList, str], class_map=None, name=None, description=None):
        super().__init__(records, class_map, name, description)
        self.data = None
        self.stats_dataset = None
        self.stats_image = None
        self.stats_class = None
    
    @staticmethod
    def parse_df_to_records(record_data_df):
        return BboxRecordDataframeParser(record_data_df).parse(RandomSplitter([1]))[0]
    
    def get_image_plot_by_id(self, index, width, height):
        return draw_record_with_bokeh(self[index], display_bbox=True, width=width, height=height, return_figure=True)

In [ ]:
test_record_dataset = BboxRecordDataset(test_valid_records, test_class_map)
assert isinstance(test_record_dataset.data, pd.DataFrame)
assert isinstance(test_record_dataset.stats_dataset, pd.DataFrame)
assert isinstance(test_record_dataset.stats_class, pd.DataFrame)
assert isinstance(test_record_dataset.stats_image, pd.DataFrame)

In [ ]:
mask = np.array([False]*test_record_dataset.data.shape[0])
mask[1] = True
masked_test_record = test_record_dataset.create_new_from_mask(test_record_dataset, mask)
assert len(masked_test_record) == 1

In [ ]:
# selection = cls_instance.data[mask]
# filepaths = np.unique(selection["filepath"])
# new_records = [record for record in cls_instance.records if record["filepath"] in filepaths]
# return cls(new_records)

In [ ]:
test_old_record_dataset_stats = test_record_dataset.stats_dataset
test_record_dataset.records.list = test_train_records
test_new_record_dataset_stats = test_record_dataset.stats_dataset
assert all(test_old_record_dataset_stats != test_new_record_dataset_stats)

In [ ]:
test_record_dataset.data

In [ ]:
test_regenerated_record_dataset = BboxRecordDataset.load_from_record_dataframe(test_record_dataset.data)
assert len(test_regenerated_record_dataset.records) == len(test_record_dataset.records)

In [ ]:
shutil.rmtree("dump_dir", ignore_errors=True)
os.mkdir("dump_dir")
test_record_dataset.name = ""
test_record_dataset.save("dump_dir")
test_record_dataset.save("dump_dir")
assert len(os.listdir("dump_dir")) == 2
assert os.path.isfile("dump_dir/dataset.json")
test_loaded_record_dataset = BboxRecordDataset("dump_dir/dataset.json")
assert test_record_dataset.data.sort_values("area").shape == test_loaded_record_dataset.data.sort_values("id").shape
shutil.rmtree("dump_dir")